# 🧙 Gandalf Chatbot - LangChain RAG with Hugging Face
This notebook builds a Retrieval-Augmented Generation (RAG) chatbot using LangChain and Hugging Face.
- Embeds a PDF into FAISS
- Loads a Hugging Face LLM endpoint
- Answers questions about the document

In [1]:
# 📦 Install Required Packages
!pip install -q langchain langchain-huggingface huggingface_hub sentence-transformers faiss-cpu python-dotenv pypdf ipywidgets

In [2]:
# 🔐 Load Environment Variables
import os
from dotenv import load_dotenv
load_dotenv()
hf_token = os.getenv("HUGGINGFACE_HUB_TOKEN")

In [4]:
# 📚 Load and Chunk the PDF
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("Tolkien-J.-The-lord-of-the-rings-HarperCollins-ebooks-2010.pdf")  # Change path to your file
pages = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(pages)
print(f"✅ Loaded and split {len(docs)} chunks.")

✅ Loaded and split 4246 chunks.


In [5]:
# 🔎 Embed Documents and Save Vectorstore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("gandalf_index")
print("✅ Vectorstore saved.")

✅ Vectorstore saved.


In [6]:
# 🤖 Load Hugging Face LLM Endpoint
from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="tiiuae/falcon-7b-instruct",  # Try other models if needed
    temperature=0.7,
    max_new_tokens=512,
    huggingfacehub_api_token=hf_token
)

In [7]:
# 🔁 Ask a Question with RAG Chain
from langchain.chains import RetrievalQA

retriever = FAISS.load_local("gandalf_index", embeddings).as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

question = "What happened in the mines of Moria?"
result = qa_chain.invoke({"query": question})

print("🧙 Gandalf says:\n", result['result'])

ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).